In [1]:
import warnings

warnings.filterwarnings(action='ignore')

# COMPAS Tutorial Code

The process is exactly same with "Tutorial.ipynb", but only difference is the structure of models. 

In [2]:
from preprocessing import Dataset
import numpy as np

np.random.seed(124)

test_size = 0.25
n_target = 10

Compas = Dataset('compas')
target_indices = np.random.choice(Compas.data.index, n_target)
(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta), tr_scaler = Compas.split_dataset(test_size, target_indices)

In [3]:
from model import SurrogateModels

model_names = ['nn_tanh_10_2','nn_relu_5_2', 'nn_relu_25_10', 'nn_identity',
               'rf_gini', 'rf_entropy', 'gb', 'ada', 'log_reg']

s_models = SurrogateModels(model_names)
s_models.train_all(x_tr, y_tr)
s_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])

train models..


100%|██████████| 9/9 [00:11<00:00,  1.25s/it]
9it [00:00, 18.00it/s]


,train acc,test acc,target acc
s-nn_tanh_10_2,0.7749,0.7576,0.6
s-nn_relu_5_2,0.7624,0.7712,0.6
s-nn_relu_25_10,0.7768,0.7680,0.7
s-nn_identity,0.7552,0.7745,0.5
s-rf_gini,0.7552,0.7641,0.5
s-rf_entropy,0.7572,0.7693,0.5
s-gb,0.7589,0.7803,0.6
s-ada,0.7559,0.7803,0.6
s-log_reg,0.7565,0.7745,0.5


In [4]:
from prob_decision_boundary import PDB

prob_dec = PDB(s_models.models)
x_all = np.concatenate([x_tr, x_te], axis=0)
prob_dec.fit_all(x_all)
sn_te_labels = prob_dec.predict(x_te)

100%|██████████| 9/9 [00:00<00:00, 81.03it/s]


In [5]:
sn_te_labels[sn_te_labels == -1] = 0
te_acc = sum(sn_te_labels==y_te)/len(y_te)
print('Test Acc: %s' % te_acc)

Test Acc: 0.7764095917044718


In [6]:
from gen_disinfos import GANcandidates

compas = Compas.data
column_cat = Compas.column_cat
column_int = Compas.column_int
columns_1hot = Compas.data_1hot.columns

gan_gen = GANcandidates()
gan_gen.fit(compas, column_cat, column_int)

In [7]:
_ = gan_gen.generate()
gan_cand_list = gan_gen.nearest_points(tr_scaler, target_indices, columns_1hot)

In [8]:
from IPython.display import display
import pandas as pd

display(pd.concat([g.iloc[[0]]for g in gan_cand_list],ignore_index=True))

,c-charge-degree,age-cat,race,sex,age,juv-fel-count,juv-misd-count,juv-other-count,priors-count,two-year-recid
0,M,Greater than 45,African-American,Female,55,0,0,0,1,0
1,F,25 - 45,Caucasian,Female,31,0,0,0,5,0
2,F,25 - 45,Hispanic,Male,26,0,0,0,0,0
3,M,25 - 45,African-American,Female,26,0,0,0,2,1
4,M,25 - 45,Caucasian,Male,37,0,0,0,0,0
5,F,25 - 45,Caucasian,Male,28,0,0,0,2,0
6,F,25 - 45,Caucasian,Male,33,0,0,0,0,0
7,M,Less than 25,African-American,Female,22,0,0,0,0,0
8,F,25 - 45,African-American,Male,28,1,0,0,3,1
9,M,25 - 45,Caucasian,Female,30,0,0,0,0,0


In [9]:
from gen_disinfos import WMcandidates, agg_disinfo
from tqdm import tqdm
  
compas_1hot = Compas.data_1hot
compas_label = Compas.label

wm_gen = WMcandidates(compas_1hot, compas_label, target_indices)
wm_cand_list = wm_gen.watermarking(tr_scaler, compas.columns, column_cat, column_int)

100%|██████████| 10/10 [00:35<00:00,  3.53s/it]


In [10]:
display(pd.concat([w.iloc[[0]]for w in wm_cand_list],ignore_index=True))

,c-charge-degree,age-cat,race,sex,age,juv-fel-count,juv-misd-count,juv-other-count,priors-count,two-year-recid
0,M,Greater than 45,African-American,Female,55,0,0,0,1,0
1,F,25 - 45,Caucasian,Female,32,0,0,0,5,0
2,F,25 - 45,Hispanic,Male,26,0,0,0,0,0
3,M,25 - 45,African-American,Female,26,0,0,0,2,1
4,M,25 - 45,Caucasian,Male,37,0,0,0,0,0
5,F,25 - 45,Caucasian,Male,28,0,0,0,2,0
6,F,25 - 45,Caucasian,Male,33,0,0,0,0,0
7,M,Less than 25,African-American,Female,22,0,0,0,0,0
8,F,25 - 45,African-American,Male,28,1,0,0,5,1
9,M,25 - 45,Caucasian,Female,30,0,0,0,0,0


In [11]:
from model import VictimModels

v_models = VictimModels()
v_models.train_all(x_tr, y_tr)
result_clean = v_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])

train models..


100%|██████████| 19/19 [00:50<00:00,  2.66s/it]
19it [00:02,  6.50it/s]


In [12]:
import pandas as pd

x_dis, y_dis = [], []
for ti in range(n_target):
    xt, yt = x_ta[ti], y_ta[ti]
    wm_cand = wm_cand_list[ti]
    gan_cand = gan_cand_list[ti]
    candidates = pd.concat((wm_cand, gan_cand))
    
    x_tmp, y_tmp = agg_disinfo(prob_dec, candidates, tr_scaler, x_tr, y_tr, xt, yt, 
                               columns_1hot, n_disinfo=100)
    x_dis.extend(x_tmp)
    y_dis.extend(y_tmp)

100%|██████████| 9/9 [00:00<00:00, 279.77it/s]


In [13]:
x_tr_dis = np.concatenate((x_tr, x_dis), axis=0)
y_tr_dis = np.concatenate((y_tr, y_dis), axis=0).astype(int)

In [14]:
v_models_dis = VictimModels()
v_models_dis.train_all(x_tr_dis, y_tr_dis)
result_dis = v_models_dis.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])

train models..


100%|██████████| 19/19 [00:55<00:00,  2.94s/it]
19it [00:03,  5.28it/s]


In [15]:
from utils import compare_result

compare_result(result_clean, result_dis)

,mean,std
train acc,-1.765789,1.188483
test acc,-2.006316,1.582902
target acc,-24.736842,16.113632


In [102]:
from sklearn.model_selection import train_test_split
from model import AttackModels, attack_input

np.random.seed(333)

vi = 1

victim_clean = v_models.models[vi]
x_mia, y_mia, x_mia_ta, y_mia_ta = attack_input(victim_clean, x_tr, y_tr, x_te, y_te, x_ta, y_ta)
x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)

In [103]:
a_models = AttackModels()
a_models.train_all(x_mia_tr, y_mia_tr)
a_result_clean = a_models.show_performance([(x_mia_tr,y_mia_tr), (x_mia_te,y_mia_te), (x_mia_ta,y_mia_ta)],
                         cnames=['train attack', 'test attack','target attack'])

train models..


100%|██████████| 19/19 [00:05<00:00,  3.25it/s]
19it [00:01, 17.62it/s]


In [104]:
victim_dis = v_models_dis.models[vi]
x_mia, y_mia, x_mia_ta, y_mia_ta = attack_input(victim_dis, x_tr, y_tr, x_te, y_te, x_ta, y_ta)
x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)

In [105]:
a_models_dis = AttackModels()
a_models_dis.train_all(x_mia_tr, y_mia_tr)
a_result_dis = a_models_dis.show_performance([(x_mia_tr,y_mia_tr), (x_mia_te,y_mia_te), (x_mia_ta,y_mia_ta)],
                         cnames=['train attack', 'test attack','target attack'])

train models..


100%|██████████| 19/19 [00:06<00:00,  2.98it/s]
19it [00:01, 16.22it/s]


In [106]:
idxs = a_result_clean['target attack acc'] >= 0.5
compare_result(a_result_clean.loc[idxs], a_result_dis.loc[idxs])

,mean,std
train attack acc,0.300000,1.035047
test attack acc,-0.675714,2.360990
target attack acc,-10.714286,9.972490
